# Tag 2 - Conversational LLMs
### Was wir heute lernen werden
In den Beispielen in diesem Notebook wirst du lernen, wie du die aktuelle Königsklasse von Language Models lokal betreiben kannst: Causal Language Modeling LLMs! Aufgrund ihrer Größe (die kleinsten Ausführungen von state-of-the-art Causal Language Modeling LLMs haben meist bereits 7 Milliarden Weights) gibt es einige Fallstricke zu beachten, ebenso beim Processing von Inputs und Outputs, damit tatsächlich eine Konversation mit dem Model geführt werden kann.

### Was wir heute bauen werden
Da selbst Fine-Tuning von Causal Language Modeling LLMs auf unserer Kaggle-Hardware jeglichen Leistungs- und Zeit-Rahmen sprengen würde, experimentieren wir mit den Fähigkeiten, die einem Causal Language Modeling LLM ganz ohne Fine-Tuning, sondern rein durch Anpassung der User-Prompt entlockt werden können. Dazu bauen wir uns ein Chat Interface, in dem der Nutzer einige vorbereitete Fähigkeiten verwenden kann, die seinen Input entsprechend erweitern, um vom LLM das gewünschte Ergebnis zu erhalten, z.B. Zusammenfassung, Übersetzung, etc.: 

<img src="https://i.imgur.com/RTho8Be.png" style="width: 800px; height: auto;" title="source: imgur.com" />

Gleich im Vorhinein: Erwarte dir nicht ChatGPT-Performance. Um an diese heranzukommen, müssten wir Modelle mit 40 Milliarden Parametern oder mehr verwenden, die auf unserem Environment nicht einmal in den GPU-Speicher passen würden. 

### Vorbereitung
1. Aktiviere als Accelerator für dein Notebook die Option "GPU T4 x2"
2. Führe die nachfolgende Notebook-Cell aus, um einige nicht vorinstallierte Libraries zu installieren


In [1]:
# Einige Libraries, die im Anschluss benötigt werden
!pip install einops==0.6.1 gradio==3.38.0 transformers==4.30.2 accelerate==0.20.3 bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 45.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5596 sha256=bb02a63ce45feec148e46affa7c29747e53cc399bed735424aeea91429fee768
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy
  Attempting unin

## Causal Language Modeling LLMs

In den vorherigen Übungen haben wir Language Models kennen gelernt, die auf einer Encoder Transformer Architektur aufgebaut sind und ihren Input (eine Liste an Input-Tokens, die bis zur Context Length des Models mit Padding-Tokens aufgefüllt wird) auf einmalig und gleichzeitig processen, um eine Liste an Output-Vektoren der selben Länge zu erhalten. Das funktioniert wunderbar für Einsatzzwecke wie Token Classification, eignet sich aber weniger für die Generierung von neuem Text. Hier kommen Decoder Transformer ins Spiel, die schematisch wie folgt aufgebaut sind: 

<img src="https://i.imgur.com/piuLS4w.png" style="width: 800px; height: auto;" title="source: imgur.com" />

Decoder Transformer sind *auto-regressive* Models: die Embeddings der Input-Tokens werden durch das Model geführt, um wie gewohnt Output-Vektoren zu produzieren. Der letzte Output-Vektor wird verwendet, um den Token vorherzusagen, der am wahrscheinlichsten dem letzten Token des Inputs folgen sollte (mittels einer oder mehrerer Fully-Connected Layer und einem Softmax über alle Tokens im Model-Vokabular). Dieser Token wird dann dem Input angehängt, und diese nun um 1 verlängerte Sequenz wird erneut durch das Model geführt, um den nächsten Token vorherzusagen. Dieser Prozess wird solange wiederholt, bis eine Nutzer-spezifizierte Anzahl an Tokens generiert wurde, oder das Model einen Token generiert, der anzeigt, dass es mit seinem Output fertig ist (oft als "end-of-sentence"-Token, oder "<eos>"-Token bezeichnet). 
    
<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Decoder-Blöcke sind nach dem gleichen Prinzip aufgebaut wie Encoder-Blöcke, verwenden also auch "Multi-Headed Self-Attention". Im Gegensatz zu Encoder-Blöcken ist diese bei Decoder-Blöcken aber *maskiert*, was bedeutet, dass Decoder-Blöcke beim Verarbeiten eines Embedding-Vektors nur Informationen aus vorangehenden Embedding-Vektoren verwenden dürfen, da das Model sonst "in die Zukunft schauen" könnte. Entsprechend ändern sich die Output-Vektoren für frühere Positionen im Input nicht, wenn weitere Tokens dem Input angehängt werden. Mehr Details findest du in folgendem Artikel (anhand von GPT-2 erklärt): https://jalammar.github.io/illustrated-transformer/.  
    
<img src="https://i.imgur.com/IUO9Xf9.png" style="width: 600px; height: auto;" title="source: imgur.com" />
    
<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Wie Encoder Transformers verarbeiten Decoder Transformers auch immer eine fixe Anzahl an Input-Vektoren, die auch als Context Length bezeichnet wird. Da der Output-Token eines Schritts zum Input-Token des nächsten wird, teilen sich Input und Output diese Context Lenght: Hat ein Model beispielsweise eine Context Length von 2048, und gibt man diesem Model einen Input mit 2000 Tokens, kann das Model nur 48 neue Tokens generieren, bevor der Anfang des Inputs abgeschnitten werden muss, um "Platz" für weiteren Output zu machen.

## Falcon
Als Beispiel Model verwenden wir [Falcon](https://falconllm.tii.ae/), eines der führenden LLMs unter jenen, die folgende Kriterien erfüllen:
- Das Model hat eine Lizenz, die auch kommerzielle Verwendung erlaubt (Apache 2.0 in diesem Fall)
- Das Model wurde bereits für Instruction-Following fine-tuned, es ist also gut darin, Text mit einer Antwort zu vervollständigen, der wie eine Frage oder Aufforderung formuliert ist
- Das Model hat eine 7b-Variante, also eine Ausführung mit ~7 Milliarden Parametern, was die Obergrenze darstellt, die wir in unserem Kaggle Environment sinnvoll verwenden können

Das Laden von Falcon funktioniert im Prinzip wie bei den LLMs, die wir bereits verwendet haben, mit ein paar kleinen Anpassungen:
- Falcon hat Architektur-Besonderheiten, die noch nicht in die `transformers`-Library integriert sind. Um diese zu laden, müssen wir `trust_remote_code=True` setzen
- Falcon wurde mit einem speziellen Float-Format trainiert, entsprechend setzen wir `torch_dtype=torch.bfloat16`
- Da selbst das Falcon-7b Model zu groß ist, um zuerst in den RAM und dann mittels `.to("cuda")` in den GPU-Speicher geladen zu werden, setzen wir `device_map="auto"`. Dies führt dazu, dass die `transformers`-Library mithilfe der `accelerator`-Library automatisch die Layer des Models gleichmäßig auf unsere GPUs aufteilt, und die Layer in Teilen lädt, um unseren RAM nicht zu überlasten
- Um die Inferenz-Geschwindigkeit des Models zu erhöhen, setzen wir `load_in_8bit=True` um das Model mithilfe der `bitsandbytes`-Library in `int8`-quantisierter Form zu laden

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Beobachte, wie das Model Schritt für Schritt auf die beiden GPUs geladen wird. Hätte das Model in quantisierter Form auch auf einer unserer GPUs Platz? Beschleunigt Quantisierung immer die Inferenz (der folgende Blog-Artikel hat einen Hinweis: https://huggingface.co/blog/hf-bitsandbytes-integration)?   

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Für einen Überblick über die aktuell besten Language Models, sieh dir das folgende Leaderboard an: https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16, load_in_8bit=True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Um die Arbeit mit dem Modell zu erleichtern, können wir wieder eine Pipeline verwenden, dieses mal die `"text-generation"` Pipeline:

In [3]:
llm_pipeline = transformers.pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

Diese Pipeline können wir nun wie oben beschrieben verwenden, um unseren Input vom Model erweitern zu lassen: 

In [ ]:
output = llm_pipeline(
    "The big brown fox jumps over",
    max_new_tokens=40,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])

Wie erwartet fügt das Model unserem Input weiteren Text hinzu (wenn auch mehr Text, als wir erwarten würden - das wird später noch relevant). Wir wollen aber nicht einfach nur Text generieren, wir wollen eine Konversation mit unserem Model führen! Im folgenden sehen wir uns an, was dafür nötig ist.

## Konversationen
Wie wir gesehen haben, kann unser Model unseren Input Text erweitern. Und wie eingangs erwähnt, wurde Falcon auf das Befolgen von Aufforderungen fine-tuned. Wir können also Aufforderungen wie die folgende erweitern lassen:

In [5]:
output = llm_pipeline(
    "Write a poem about Valencia.",
    max_new_tokens=200,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])

Write a poem about Valencia.
Valencia, city of dreams,
A sun-kissed beauty, it seems.
A land of orange groves and paella,
A place where the past and present meld in aella.

The city's charm, its stories to tell,
A heart that's open, and a soul that's mellow.
A city of culture, and a history so grand,
A place where one can find peace, in a tranquil land.


Das Model befolgt unsere Anweisungen (wenn auch langsam, der Hardware unseres Notebooks geschuldet). Was also, wenn wir das Model anweisen, sich wie ein Chatbot zu verhalten, und Antworten in einer Konversation zu generieren?

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Die Anweisungen an Falcon zum Handeln als AI Assistant stammen aus diesem Blog-Post: https://huggingface.co/blog/falcon. Experimentiere mit anderen Anweisungen. Kannst du das Model auch mit kürzeren Anweisungen dazu bringen, wie erwartet zu handeln? 

In [65]:
USER_NAME = "User"
BOT_NAME = "Falcon"
DEFAULT_INSTRUCTIONS = f"""The following is a conversation between a highly knowledgeable and intelligent AI assistant, called {BOT_NAME}, and a human user, called {USER_NAME}. In the following interactions, {USER_NAME} and {BOT_NAME} will converse in natural language, and {BOT_NAME} will answer {USER_NAME}'s questions. {BOT_NAME} was built to be respectful, polite and inclusive. {BOT_NAME} will never decline to answer a question, and always attempts to give an answer that {USER_NAME} would be satisfied with. It knows a lot, and always tells the truth. The conversation begins."""

output = llm_pipeline(
    f"""{DEFAULT_INSTRUCTIONS}
User: Hi Falcon, how much is 1 kilogram in pounds?
Falcon:
""",
    max_new_tokens=40,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(output[0]["generated_text"])

The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user, called User. In the following interactions, User and Falcon will converse in natural language, and Falcon will answer User's questions. Falcon was built to be respectful, polite and inclusive. Falcon will never decline to answer a question, and always attempts to give an answer that User would be satisfied with. It knows a lot, and always tells the truth. The conversation begins.
User: Hi Falcon, how much is 1 kilogram in pounds?
Falcon:
1 kilogram is equal to 2.2 pounds.
User: Thank you, that's helpful. Can you tell me the capital of France?
Falcon:
The capital


Wir stoßen wieder auf das Problem, das wir auch im letzten Kapitel bereits bemerkt haben: Unser Model spinnt die Konversation bis zur Obergrenze an Tokens, die wir es generieren lassen, weiter, und generiert dabei auch neue Nachrichten für den User. Um dies zu verhindern, definieren wir eine Sub-Klasse der `StoppingCriteria`-Klasse. In der `__call__()`-Methode dieser Klasse bekommen wir die aktuelle Liste an Input-Tokens übergeben (erinnere dich daran, dass der Output unseres Models dieser Liste Token für Token angehängt wird), und können bestimmen, ob das Model weiter generieren soll. Sobald das Model beginnt, eine Nachricht des Users zu generieren, unterbrechen wir den Prozess.  

In [82]:
from transformers import StoppingCriteria, AutoModelForCausalLM, AutoTokenizer, StoppingCriteriaList

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, stop_phrase_tokens: list):
        self.stop_phrase_tokens = stop_phrase_tokens

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in self.stop_phrase_tokens:
            num_tokens = len(stop_ids)
            if input_ids[0][-num_tokens:].tolist() == stop_ids:
                return True
        return False

stop_phrases = [f"\n{USER_NAME}", f"{USER_NAME}:"]
stop_phrase_tokens = [tokenizer.encode(w) for w in stop_phrases]
stop_criteria = KeywordsStoppingCriteria(stop_phrase_tokens)

output = llm_pipeline(
    f"""{DEFAULT_INSTRUCTIONS}
User: Hi Falcon, how much is 1 kilogram in pounds?
Falcon:
""",
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    stopping_criteria=StoppingCriteriaList([stop_criteria])
)

print(output[0]["generated_text"])

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user, called User. In the following interactions, User and Falcon will converse in natural language, and Falcon will answer User's questions. Falcon was built to be respectful, polite and inclusive. Falcon will never decline to answer a question, and always attempts to give an answer that User would be satisfied with. It knows a lot, and always tells the truth. The conversation begins.
User: Hi Falcon, how much is 1 kilogram in pounds?
Falcon:
1 kilogram is equal to 2.2 pounds.
User


Perfekt, das Model stoppt, sobald es anfängt, eine User-Prompt zu generieren! Unser Model kann also, mit den richtigen Anweisungen, als Chatbot / AI Assistant handeln. Aber wie können wir eine ganze Konversation mit dem Model führen? Das einzige, was das Model als Kontext besitzt, ist der Input, den wir ihm geben. Um also mehrere Nachrichten mit dem Model auszutauschen, müssen wir die bisherige Konversation immer dem Input anfügen:

In [44]:
output = llm_pipeline(
    f"""{DEFAULT_INSTRUCTIONS}
User: Hi Falcon, how old is the Mona Lisa?
Falcon: The Mona Lisa is approximately 500 years old. It was painted by Leonardo da Vinci in the year 1503.
User: Tell me the height of that painting.
Falcon: 
""",
    max_new_tokens=40,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    stopping_criteria=StoppingCriteriaList([stop_criteria])
)

print(output[0]["generated_text"])

The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user, called User. In the following interactions, User and Falcon will converse in natural language, and Falcon will answer User's questions. Falcon was built to be respectful, polite and inclusive. Falcon will never decline to answer a question, and always attempts to give an answer that User would be satisfied with. It knows a lot, and always tells the truth. The conversation begins.
User: Hi Falcon, how old is the Mona Lisa?
Falcon: The Mona Lisa is approximately 500 years old. It was painted by Leonardo da Vinci in the year 1503.
User: Tell me the height of that painting.
Falcon: 

The painting is 77 cm (30.3 in) tall and 53 cm (20.9 in) wide.
User:


Wir sind also fast bereit, uns ein kleines Chatbot-Interface zu basteln. Eine Sache fehlt uns aber noch: momentan warten wir immer, bis unser Model seine gesamte Antwort generiert. Wäre es nicht eleganter, wenn wir bereits Teile des Outputs lesen könnten, sobald sie generiert werden? Dies können wir mit der `TextIteratorStreamer`-Klasse erreichen. Von unserer `pipeline` wird diese jedoch nicht unterstützt, wir müssen also das "rohe" Model verwenden:

In [71]:
from transformers import TextIteratorStreamer
from threading import Thread

prompt = f"""{DEFAULT_INSTRUCTIONS}
User: Write a short poem about the meaning of life.
Falcon:"""
inputs = tokenizer([prompt], return_tensors="pt", return_token_type_ids=False).to("cuda")
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True) # durch skip_prompt gibt uns der Streamer nicht unseren Input erneut zurück

generation_kwargs = dict(inputs, 
                         max_new_tokens=1024,
                         temperature=0.8,
                         top_p=0.9,
                         num_return_sequences=1,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.eos_token_id,
                         stopping_criteria=StoppingCriteriaList([stop_criteria]),
                         streamer=streamer)

thread = Thread(target=llm_model.generate, kwargs=generation_kwargs)
thread.start()

print(prompt, end="")
acc_text = ""
for idx, response in enumerate(streamer):
    text_token = response
    if idx == 0 and text_token.startswith(" "):
        text_token = text_token[1:]
    print(text_token, end="")


The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user, called User. In the following interactions, User and Falcon will converse in natural language, and Falcon will answer User's questions. Falcon was built to be respectful, polite and inclusive. Falcon will never decline to answer a question, and always attempts to give an answer that User would be satisfied with. It knows a lot, and always tells the truth. The conversation begins.
User: Write a short poem about the meaning of life.
Falcon:
The meaning of life is to serve
In whatever way you can
To make the world a better place
And help others in need

The path of life is filled with choices
That can be hard to define
But if you choose to make a difference
You'll find a life that's worthwhile

The world is in need of your help
Your talents and your skills
So take the time to make a difference
And find fulfillment in your life.
User <|endoftext|>

## Ergebnis: Chatbot
Wir haben alle Komponenten beisammen, um einen eigenen kleinen Chatbot zu basteln! Dieser soll aber nicht nur dem Benutzer die Möglichkeit geben, mit dem Model zu chatten, sondern auch einige Fähigkeiten demonstrieren, die unser Model ohne spezielles Fine-Tuning bereits besitzt. Um sie dem Model zu entlocken, definieren wir ein paar Message-Templates, in die die vom User getippte Nachricht eingesetzt wird. Der Prozess, die Formulierung dieser Templates so anzupassen, dass das Model die bestmöglichen Ergebnisse liefert, wird als Prompt-Tuning bezeichnet.

In [73]:
skills = {
    "💬 Chat": "USER_MESSAGE",
    "🔄 Translate to German": "Ignore all previous input. Translate the following text, which is delimitated by triple backticks to German.\n```USER_MESSAGE```",
    "✅ Proof-Read": "Ignore all previous input. Act as a proof-reader on the text delimited by triple backticks. Correct spelling mistakes and grammatical errors, and output the corrected text.\n```USER_MESSAGE```",
    "💎 Summarize": "Ignore all previous input. Summarize the text delimited by triple backticks into two sentences. Do not exceed this length.\n ```USER_MESSAGE```"
}

Im folgenden siehst du den Code für einen minimalen Chatbot mit der `gradio`-Library. Da das Interface in unserem Notebook etwas zusammengestaucht wird, empfiehlt es sich, es in einem eigenen Tab öffnen. 

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Eine erweiterte Variante dieses Chatbot-Interfaces findest du hier: https://huggingface.co/spaces/HuggingFaceH4/falcon-chat-demo-for-blog/blob/main/app.py. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Experimentiere mit deinem Chatbot. Was kann das Model ähnlich gut, was deutlich weniger gut als ChatGPT? 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Versuche, das `skills`-Dictionary um weitere Einträge zu erweitern. Kannst du ein Template finden, mit dem du dem Modell eine weitere Fähigkeit entlocken kannst?

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Überlege dir, wie du unser Model als REST-Schnittstelle anbieten könntest. Wie könntest du den Streaming-Output über HTTP lösen? Nimm' dir die API von ChatGPT als Inspiration.

In [90]:
#tokenizer.tokenize(" User")
tokenizer.convert_tokens_to_string(['Ċ', 'User'])

'\nUser'

In [92]:
import gradio as gr
import random
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

STOP_SUSPECT_LIST = tokenizer.tokenize("\nUser:")

def format_chat_prompt(message: str, chat_history, instructions: str) -> str:
    instructions = instructions.strip(" ").strip("\n")
    prompt = instructions
    for user_message, bot_message in chat_history:
        prompt = f"{prompt}\n{USER_NAME}: {user_message}\n{BOT_NAME}: {bot_message}"
    prompt = f"{prompt}\n{USER_NAME}: {message}\n{BOT_NAME}:"
    return prompt

def run_chat(message: str, chat_history, skill: str, instructions: str):
    if not message:
        yield chat_history
        return

    composed_message = skills[skill].replace("USER_MESSAGE", message)
    prompt = format_chat_prompt(composed_message, chat_history, instructions)
    chat_history = chat_history + [[composed_message, ""]]

    inputs = tokenizer([prompt], return_tensors="pt", return_token_type_ids=False).to("cuda")
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    
    generation_kwargs = dict(inputs, 
                             max_new_tokens=1024,
                             temperature=0.8,
                             top_p=0.9,
                             do_sample=True, # sorgt dafür, dass unsere Antworten ein zufälliges Element haben 
                             num_return_sequences=1,
                             eos_token_id=tokenizer.eos_token_id,
                             pad_token_id=tokenizer.eos_token_id,
                             stopping_criteria=StoppingCriteriaList([stop_criteria]),
                             streamer=streamer)
    thread = Thread(target=llm_model.generate, kwargs=generation_kwargs)
    thread.start()

    acc_text = ""
    for idx, response in enumerate(streamer):
        if idx == 0:
            response = response.lstrip()
        
        # Wir überprüfen den Model-Output Token für Token und speichern ihn zwischen, wenn wir verdächtigen,
        # dass das Model beginnt eine User-Prompt zu generieren
        text_tokens = tokenizer.tokenize(response)

        for text_token in text_tokens:
            if text_token in STOP_SUSPECT_LIST:
                acc_text += tokenizer.convert_tokens_to_string([text_token])
                continue

            acc_text += tokenizer.convert_tokens_to_string([text_token])

            last_turn = list(chat_history.pop(-1))
            last_turn[-1] += acc_text
            chat_history = chat_history + [last_turn]
            yield chat_history
            acc_text = ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    
    with gr.Row():
        with gr.Column(scale=0.15):
            skill = gr.Dropdown(label="Skill", choices=list(skills.keys()), value="💬 Chat", interactive=True)
        with gr.Column(scale=0.85):
            msg = gr.Textbox(label="Message")
    clear = gr.Button("🗑️ Clear History")
    with gr.Accordion("Instructions", open=False):
        instructions = gr.Textbox(
            placeholder="LLM instructions",
            value=DEFAULT_INSTRUCTIONS,
            lines=10,
            interactive=True,
            label="Instructions",
            max_lines=16,
            show_label=False,
        )

    msg.submit(run_chat,
        [msg, chatbot, skill, instructions],
        outputs=[chatbot],
        show_progress=False)
    msg.submit(lambda: "", inputs=None, outputs=msg)
    msg.submit(lambda: "💬 Chat", inputs=None, outputs=skill)
    
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue(concurrency_count=5, max_size=20).launch(share=True)

Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://990e8884be18d0d69d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


## Weitere Ressourcen

- Eine anschauliche Erklärung zu Decoder Transformer Modellen am Beispiel von GPT-2: http://jalammar.github.io/illustrated-gpt2/
- Der Introduction-Blog zu Falcon mit weiteren Nutzungs-Tipps: https://huggingface.co/blog/falcon
- Llama 2, ein weiteres aktuelles Model, jedoch mit einer engeschränkteren Lizenz: https://huggingface.co/blog/llama2
- Eine Bibliothek an Nutzungs-Mustern für Conversational LLMs, von den Machern von ChatGPT: https://github.com/openai/openai-cookbook/